In [1]:
import pyodbc as odbc
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
driver_name = "ODBC Driver 17 for SQL Server"

server_name = "localhost\\SQLEXPRESS"
Database_name = 'HotelData'

connection = f"""
    DRIVER={{{driver_name}}};
    SERVER={server_name};
    DATABASE={Database_name};
    Trusted_Connection=yes;
"""

connect_hotel = odbc.connect(connection)
print(connect_hotel)

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [3]:
# List all tables in the database to verify the existence of 'HotelRooms'
cursor = connect_hotel.cursor()
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = cursor.fetchall()

# Print the list of tables
print("Tables in the database:")
for table in tables:
	print(table[0])

# Uncomment the following lines if 'HotelRooms' exists and you want to query it
# cursor.execute("SELECT * FROM HotelRooms")
# rows = cursor.fetchall()
# for row in rows:
#     print(row)



In [4]:
country_gender_count ="""
            
            SELECT *
FROM dbo.TicketActivities ta
left join dbo.Tickets t
on t.[Ticket ID] = ta.[Ticket ID]
left join dbo.ReservationActivities ra 
on t.[Reservation ID] = ra.[Reservation ID]
left join dbo.Reservations r
on r.[Reservation ID] = ra.[Reservation ID]
left join dbo.HotelRooms hr 
on hr.[Room Number] = r.[Room Number]
left join dbo.Customers c
on c.[Customer Number] = r.[Customer Number]

            """
df = pd.read_sql(country_gender_count, connect_hotel)
# connect_hotel.close()



In [24]:
null_rows = df[df.isnull().any(axis=1)]
print(null_rows)

In [5]:
Group_query = """
SELECT TOP(11) c.[Country of Origin], COUNT(c.Gender) AS gender
FROM dbo.TicketActivities ta
LEFT JOIN dbo.Tickets t
ON t.[Ticket ID] = ta.[Ticket ID]
LEFT JOIN dbo.ReservationActivities ra 
ON t.[Reservation ID] = ra.[Reservation ID]
LEFT JOIN dbo.Reservations r
ON r.[Reservation ID] = ra.[Reservation ID]
LEFT JOIN dbo.HotelRooms hr 
ON hr.[Room Number] = r.[Room Number]
LEFT JOIN dbo.Customers c
ON c.[Customer Number] = r.[Customer Number]
GROUP BY c.[Country of Origin]
ORDER BY gender DESC
"""

df_group = pd.read_sql(Group_query, connect_hotel)
df_group.plot(kind='bar', x='Country of Origin')
plt.title('Top 11 Countries by Gender Count')
plt.xlabel('Country of Origin')
plt.ylabel('Gender Count')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout for better readability
plt.show()

## All questions with codes

In [ ]:

# --1. How can the relationships among all the tables in a dataset be identified and
# --analyzed??
# SELECT *
# FROM dbo.TicketActivities ta
# inner join dbo.Tickets t
# on t.[Ticket ID] = ta.[Ticket ID]
# inner join dbo.ReservationActivities ra 
# on t.[Reservation ID] = ra.[Reservation ID]
# inner join dbo.Reservations r
# on r.[Reservation ID] = ra.[Reservation ID]
# inner join dbo.HotelRooms hr 
# on hr.[Room Number] = r.[Room Number]
# inner join dbo.Customers c
# on c.[Customer Number] = r.[Customer Number]
# --2. How can the total number of unique customers be determined?
# SELECT count(distinct [Customer Number]) Unique_customers
# FROM Customers


# --3. What steps can be taken to calculate the occupancy rate?

# SELECT 
#     (SUM(r.Nights) * 100.0) / (COUNT(*) * 30) AS Occupancy_Rate
# FROM dbo.Reservations r
# JOIN dbo.HotelRooms hr 
# ON r.[Room Number] = hr.[Room Number];

# --4. How can the distribution of customer complaints be analyzed room-wise?
# SELECT r.[Room Number], COUNT(t.[Ticket ID]) AS Total_Complaints
# FROM Tickets t
# JOIN Reservations r ON t.[Reservation ID] = r.[Reservation ID]
# GROUP BY r.[Room Number]
# ORDER BY Total_Complaints DESC;



# --5. What are the key complaints identified in customer feedback?
# SELECT top(10) Description, COUNT(*) AS Complaint_Count
# FROM Tickets
# GROUP BY Description
# ORDER BY Complaint_Count DESC;




# --6. How can the customer retention rate be calculated?

# SELECT 
#     (COUNT(DISTINCT r.[Customer Number]) * 100.0) / 
#     (SELECT COUNT(DISTINCT [Customer Number]) FROM Customers) AS Retention_Rate
# FROM Reservations r
# WHERE r.[Customer Number] IN (
#     SELECT [Customer Number]
#     FROM Reservations
#     GROUP BY [Customer Number]
#     HAVING COUNT([Reservation ID]) > 1
# );

# --7. How can the total number of complaints be identified for each room?
# SELECT  r.[Room Number], COUNT(t.[Ticket ID]) AS Total_Complaints
# FROM dbo.Tickets t
# JOIN dbo.Reservations r ON t.[Reservation ID] = r.[Reservation ID]
# GROUP BY r.[Room Number]
# ORDER BY Total_Complaints DESC;

# --8. What is the region-wise breakdown of offers sent and customer engagement?

# SELECT Residence, COUNT([Reservation ID]) AS Total_Bookings
# FROM Reservations r
# JOIN Customers c ON r.[Customer Number] = c.[Customer Number]
# GROUP BY Residence
# ORDER BY Total_Bookings DESC;

In [8]:
query = connect_hotel.cursor()
query.execute("""
SELECT 
    (SUM(r.Nights) * 100.0) / (COUNT(*) * 30) AS Occupancy_Rate
FROM dbo.Reservations r
JOIN dbo.HotelRooms hr 
ON r.[Room Number] = hr.[Room Number];


""") 
query.fetchall()

In [6]:
query = connect_hotel.cursor()
query.execute("""

select r.Age, r.[Room Category], c.[First Name], c.[Last Name]
from dbo.Reservations r
inner join dbo.Customers c
on r.[Customer Number] = c.[Customer Number]
-- where r.[Room Category] = 'Double Room'
order by r.[Room Category] desc

""") 
query.fetchall()